In [1]:
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
import torch.nn.functional as F
from torch import nn
import pandas as pd
import numpy as np
import random
import torch
import math
import os

In [2]:
seed = 27
batch_size = 1
vocab_size = 50_000
max_seq_len = 4000
device = "cuda"

In [3]:
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [4]:
train = pd.read_json('/kaggle/input/pii-detection-removal-from-educational-data/train.json')
test = pd.read_json('/kaggle/input/pii-detection-removal-from-educational-data/test.json')

In [5]:
def build_vocabulary(corpus):
    tokenizer = get_tokenizer("spacy", language="en_core_web_lg")

    def tokenizer_fn(data_iterator):
        for text in data_iterator:
            yield tokenizer(text)

    vocab = build_vocab_from_iterator(tokenizer_fn(corpus), specials=["<unk>"], max_tokens=vocab_size)
    vocab.set_default_index(vocab["<unk>"])

    return vocab, tokenizer

corpus = list(train["full_text"].values) + list(test["full_text"].values)
vocab, tokenizer = build_vocabulary(corpus)

In [6]:
def tokenize_text(dataframe, vocab, is_test=False):
    dataframe['tokens'] = dataframe['tokens'].apply(lambda x: x[:max_seq_len])
    dataframe['seq_len'] = dataframe['tokens'].apply(lambda x: len(x))
    dataframe["token_num"] = dataframe["tokens"].apply(lambda x: np.array(vocab(x), dtype=np.int64))
    dataframe['seq_len'] = dataframe['tokens'].apply(lambda x: len(x))
    percentiles = [i * 0.1 for i in range(10)] + [.95, .99, .995]
    buckets = np.quantile(dataframe['seq_len'], percentiles)
    bucket_labels = [i for i in range(len(buckets) - 1)]
    dataframe['bucket'] = pd.cut(dataframe['seq_len'], bins=buckets, labels=bucket_labels)
    dataframe["bucket"] = dataframe["bucket"].fillna(0)
    dataframe["bucket"] = dataframe["bucket"].astype(int)
    dataframe["seq_len"] = dataframe["seq_len"].astype(int)
    if is_test:
        return dataframe[['token_num', 'seq_len', 'bucket', 'document']]
    return dataframe[['token_num', 'labels', "seq_len", "bucket"]]

In [7]:
test = tokenize_text(test, vocab, True)

In [8]:
label_2_id = {
    "O": 0,
    "B-NAME_STUDENT": 1,
    "I-NAME_STUDENT": 2,
    "B-URL_PERSONAL": 3,
    "B-ID_NUM": 4,
    "B-EMAIL": 5,
    "I-STREET_ADDRESS": 6,
    "I-PHONE_NUM": 7,
    "B-USERNAME": 8,
    "B-PHONE_NUM": 9,
    "B-STREET_ADDRESS": 10,
    "I-URL_PERSONAL": 11,
    "I-ID_NUM": 12
}

id_2_label = {v: k for k, v in label_2_id.items()}

In [9]:
def complete_batch(dataframe, batch_size):
    complete_buckets = []
    buckets = [bucket_df for _, bucket_df in dataframe.groupby('bucket')]

    for gr_id, bucket in enumerate(buckets):
        l = len(bucket)
        remainder = l % batch_size
        integer = l // batch_size

        if remainder != 0:
            bucket = pd.concat([bucket, pd.concat([bucket.iloc[:1]] * (batch_size - remainder))], ignore_index=True)
            integer += 1

        batch_ids = []
        for i in range(integer):
            batch_ids.extend([f'{i}_bucket{gr_id}'] * batch_size)

        bucket['batch_id'] = batch_ids
        complete_buckets.append(bucket)
    return pd.concat(complete_buckets, ignore_index=True)


def shuffle_batches(dataframe):
    batch_buckets = [df_new for _, df_new in dataframe.groupby('batch_id')]
    random.shuffle(batch_buckets)
    return pd.concat(batch_buckets).reset_index(drop=True)


def concater_collate(batch):
    (xx, yy, lengths, buckets) = zip(*batch)
    xx = torch.cat(xx, 0)
    yy = torch.from_numpy(np.array(yy))
    return xx, yy, list(lengths), list(buckets)

def concater_collate_test(batch):
    (xx, lengths, buckets, documents) = zip(*batch)
    xx = torch.cat(xx, 0)
    return xx, list(lengths), list(buckets), list(documents)


class PIIDataset(Dataset):
    def __init__(self, dataframe, batch_size, is_test=False):
        dataframe = complete_batch(dataframe=dataframe, batch_size=batch_size)
        dataframe = shuffle_batches(dataframe=dataframe)
        self.is_test = is_test
        if is_test:
            self.dataframe = dataframe[['token_num', 'seq_len', 'bucket', 'document']]
        else:
            self.dataframe = dataframe[['token_num', 'labels', 'seq_len', 'bucket']]

    def __getitem__(self, index):
        if self.is_test:
            X, seq_len, bucket, document = self.dataframe.iloc[index, :]
            X = torch.from_numpy(X)
            return X, seq_len, bucket, document
             
        else:
            X, Y, seq_len, bucket = self.dataframe.iloc[index, :]
            Y = torch.from_numpy(np.array(Y))
            padding = max_seq_len - len(Y)
            Y = F.pad(Y, (0, padding), value=99)
            X = torch.from_numpy(X)
            return X, Y, seq_len, bucket

    def __len__(self):
        return len(self.dataframe)

In [10]:
test_dataset = PIIDataset(test, batch_size, True)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=concater_collate_test,
    shuffle=False,
    drop_last=False,
    num_workers=4
)

In [11]:
class MLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class RotateChord(nn.Module):
    def __init__(self, n_tracks, track_size):
        super(RotateChord, self).__init__()
        self.n_tracks = n_tracks
        self.track_size = track_size

    def forward(self, x, lengths):

        ys = torch.split(
            tensor=x,
            split_size_or_sections=lengths,
            dim=0
        )

        zs = []

        for y in ys:
            y = torch.split(
                tensor=y,
                split_size_or_sections=self.track_size,
                dim=-1
            )
            z = [y[0]]
            for i in range(1, len(y)):
                offset = -2 ** (i - 1)
                z.append(torch.roll(y[i], shifts=offset, dims=0))
            z = torch.cat(z, -1)
            zs.append(z)

        z = torch.cat(zs, 0)
        assert z.shape == x.shape, 'shape mismatch'
        return z


class ChordMixerBlock(nn.Module):
    def __init__(
            self,
            embedding_size,
            n_tracks,
            track_size,
            hidden_size,
            mlp_dropout,
            layer_dropout
    ):
        super(ChordMixerBlock, self).__init__()

        self.mixer = MLP(
            embedding_size,
            hidden_size,
            embedding_size,
            act_layer=nn.GELU,
            drop=mlp_dropout
        )

        self.dropout = nn.Dropout(layer_dropout)

        self.rotator = RotateChord(n_tracks, track_size)

    def forward(self, data, lengths):
        res_con = data
        data = self.mixer(data)
        data = self.dropout(data)
        data = self.rotator(data, lengths)
        data = data + res_con
        return data


class ChordMixer(nn.Module):
    def __init__(self, vocab_size=vocab_size, max_seq_len=max_seq_len, track_size=16, hidden_size=196, mlp_dropout=0.0, layer_dropout=0.0):
        super(ChordMixer, self).__init__()
        self.max_n_layers = math.ceil(np.log2(max_seq_len))
        n_tracks = math.ceil(np.log2(max_seq_len))
        embedding_size = int(n_tracks * track_size)
        self.embedding_size = embedding_size
        self.max_seq_len = max_seq_len
        
        self.embedding = nn.Embedding(
            vocab_size,
            embedding_size
        )

        self.chordmixer_blocks = nn.ModuleList(
            [
                ChordMixerBlock(
                    embedding_size,
                    n_tracks,
                    track_size,
                    hidden_size,
                    mlp_dropout,
                    layer_dropout
                )
                for _ in range(self.max_n_layers)
            ]
        )

        self.reshape = nn.Linear(embedding_size, max_seq_len * embedding_size)
        self.final = nn.Linear(embedding_size, 13)

    def forward(self, data, lengths):
        n_layers = math.ceil(np.log2(lengths[0]))

        data = self.embedding(data)
        for layer in range(n_layers):
            data = self.chordmixer_blocks[layer](data, lengths)

        data = [torch.mean(t, dim=0) for t in torch.split(data, lengths)]
        data = torch.stack(data)
        
        data = self.reshape(data)
        data = data.view(data.size(0), self.max_seq_len, -1)
        
        data = self.final(data)
        
        return data        

In [12]:
path = "/kaggle/input/pii-detection-chordmixer/checkpoints/epoch_70_loss_5.039997501475992e-05.pt"
model = ChordMixer().to(device)
model.load_state_dict(torch.load(path))
model.eval()

predictions = []
for x, seq_len, bucket, document in test_dataloader:
    x = x.to(device)
    
    y_hat = model(x, seq_len)
    y_pred = torch.argmax(y_hat, dim=2)
    temp_preds = {
        "tokens": x.cpu().numpy(),
        "document": document,
        "labels": y_pred.cpu().numpy()
    }
    predictions.append(temp_preds)

In [13]:
result = []
for pred in predictions:
    documents = pred['document']
    labels = pred['labels']
    tokens = pred['tokens']
    for i in range(batch_size):
        document = documents[i]
        for idx, label in enumerate(labels[i]):
            if label != 0 and idx < len(tokens):
                result.append({
                    "document": document,
                    "token": idx,
                    "word": vocab.get_itos()[tokens[idx]],
                    "label": id_2_label[label]
                })

In [14]:
result_df = pd.DataFrame(result)
result_df = result_df.drop_duplicates(subset=["document", "token", "label"])
result_df = result_df.sort_values(by=['document', 'token']).reset_index(drop=True)
result_df["row_id"] = list(range(len(result_df)))
result_df = result_df[["row_id", "document", "token", "label"]]
result_df = result_df.astype(
    {
        "row_id": "int64",
        "document": "int64",
        "token": "int64",
        "label": "string",
    }
)
result_df.to_csv("submission.csv", index=False)
result_df

,row_id,document,token,label
0,0,7,9,B-NAME_STUDENT
1,1,7,10,I-NAME_STUDENT
2,2,7,482,B-NAME_STUDENT
3,3,7,483,I-NAME_STUDENT
4,4,7,741,B-NAME_STUDENT
5,5,7,742,I-NAME_STUDENT
6,6,10,0,B-NAME_STUDENT
7,7,10,1,I-NAME_STUDENT
8,8,10,464,B-NAME_STUDENT
9,9,10,465,I-NAME_STUDENT
